In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import nltk
from nltk.corpus import stopwords

In [2]:
tweets=pd.read_json('../Datos/Dataset for Detection of Cyber-Trolls.json',lines=True)
tweets=tweets.drop('extras',axis=1)
tweets

,content,annotation
0,Get fucking real dude.,"{'notes': '', 'label': ['1']}"
1,She is as dirty as they come and that crook ...,"{'notes': '', 'label': ['1']}"
2,why did you fuck it up. I could do it all day...,"{'notes': '', 'label': ['1']}"
3,Dude they dont finish enclosing the fucking s...,"{'notes': '', 'label': ['1']}"
4,WTF are you talking about Men? No men thats n...,"{'notes': '', 'label': ['1']}"
...,...,...
19996,I dont. But what is complaining about it goi...,"{'notes': '', 'label': ['0']}"
19997,Bahah yeah i&;m totally just gonna&; get pis...,"{'notes': '', 'label': ['0']}"
19998,hahahahaha >:) im evil mwahahahahahahahahaha,"{'notes': '', 'label': ['0']}"
19999,What&;s something unique about Ohio? :),"{'notes': '', 'label': ['0']}"


In [3]:
tweets['content']=tweets.content.str.lower()
tweets.head()

,content,annotation
0,get fucking real dude.,"{'notes': '', 'label': ['1']}"
1,she is as dirty as they come and that crook ...,"{'notes': '', 'label': ['1']}"
2,why did you fuck it up. i could do it all day...,"{'notes': '', 'label': ['1']}"
3,dude they dont finish enclosing the fucking s...,"{'notes': '', 'label': ['1']}"
4,wtf are you talking about men? no men thats n...,"{'notes': '', 'label': ['1']}"


In [4]:
tweets['clasificacion']=tweets.annotation.str.get('label')
tweets=tweets.drop('annotation',axis=1)
tweets['clasificacion']=tweets.clasificacion.apply(lambda x:x[0])
tweets['clasificacion']=tweets.clasificacion.astype(int)
tweets

,content,clasificacion
0,get fucking real dude.,1
1,she is as dirty as they come and that crook ...,1
2,why did you fuck it up. i could do it all day...,1
3,dude they dont finish enclosing the fucking s...,1
4,wtf are you talking about men? no men thats n...,1
...,...,...
19996,i dont. but what is complaining about it goi...,0
19997,bahah yeah i&;m totally just gonna&; get pis...,0
19998,hahahahaha >:) im evil mwahahahahahahahahaha,0
19999,what&;s something unique about ohio? :),0


In [5]:
tweets['content_cleaned'] = tweets['content'].apply(lambda text: re.sub(r"[^a-zA-Z0-9\s]", "", text))
tweets

,content,clasificacion,content_cleaned
0,get fucking real dude.,1,get fucking real dude
1,she is as dirty as they come and that crook ...,1,she is as dirty as they come and that crook ...
2,why did you fuck it up. i could do it all day...,1,why did you fuck it up i could do it all day ...
3,dude they dont finish enclosing the fucking s...,1,dude they dont finish enclosing the fucking s...
4,wtf are you talking about men? no men thats n...,1,wtf are you talking about men no men thats no...
...,...,...,...
19996,i dont. but what is complaining about it goi...,0,i dont but what is complaining about it goin...
19997,bahah yeah i&;m totally just gonna&; get pis...,0,bahah yeah im totally just gonna get pissed ...
19998,hahahahaha >:) im evil mwahahahahahahahahaha,0,hahahahaha im evil mwahahahahahahahahaha
19999,what&;s something unique about ohio? :),0,whats something unique about ohio


In [6]:
tweets=tweets.drop('content',axis=1)
tweets

,clasificacion,content_cleaned
0,1,get fucking real dude
1,1,she is as dirty as they come and that crook ...
2,1,why did you fuck it up i could do it all day ...
3,1,dude they dont finish enclosing the fucking s...
4,1,wtf are you talking about men no men thats no...
...,...,...
19996,0,i dont but what is complaining about it goin...
19997,0,bahah yeah im totally just gonna get pissed ...
19998,0,hahahahaha im evil mwahahahahahahahahaha
19999,0,whats something unique about ohio


In [7]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/davidhincapie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def remove_stopwords(text):
    words=text.split()
    filtered_words=[word for word in words if word.lower() not in stop_words]

    return ' '.join(filtered_words)

In [9]:
tweets['content_no_stopwords'] = tweets['content_cleaned'].apply(remove_stopwords)
tweets

,clasificacion,content_cleaned,content_no_stopwords
0,1,get fucking real dude,get fucking real dude
1,1,she is as dirty as they come and that crook ...,dirty come crook rengel dems fucking corrupt j...
2,1,why did you fuck it up i could do it all day ...,fuck could day lets hour ping later sched writ...
3,1,dude they dont finish enclosing the fucking s...,dude dont finish enclosing fucking showers hat...
4,1,wtf are you talking about men no men thats no...,wtf talking men men thats menage thats gay
...,...,...,...
19996,0,i dont but what is complaining about it goin...,dont complaining going
19997,0,bahah yeah im totally just gonna get pissed ...,bahah yeah im totally gonna get pissed talking...
19998,0,hahahahaha im evil mwahahahahahahahahaha,hahahahaha im evil mwahahahahahahahahaha
19999,0,whats something unique about ohio,whats something unique ohio


In [10]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweets['content_no_stopwords'])
secuencias = tokenizer.texts_to_sequences(tweets['content_no_stopwords'])
vocabulario_size = len(tokenizer.word_index) + 1
secuencias

[[10, 11, 186, 78],
 [714, 97, 4578, 4579, 4580, 11, 3074, 622, 39, 3642, 90, 7],
 [6, 48, 53, 311, 432, 2331, 450, 4581, 696, 360],
 [78, 9, 1069, 4582, 11, 1744, 1, 371, 4583, 900, 67, 4584, 777, 129, 534],
 [249, 247, 678, 678, 21, 4585, 21, 22],
 [58, 574, 778, 558, 407, 82, 1307, 6, 1745, 1596, 263, 49, 88],
 [4, 283, 4586, 4587, 72, 3643, 9, 52, 1901, 1746, 6, 78, 286, 16],
 [28, 1122, 4588, 4589, 59, 4590, 1308, 2651, 148, 14, 575, 4591],
 [91, 4, 349, 51, 801, 22, 100, 715, 4592, 23, 53, 9, 31, 210, 588],
 [8, 95, 98, 1902, 1309, 22, 100, 2652, 801],
 [971, 3075, 100, 3, 1597, 2332, 4593, 69, 19, 172, 4594, 4595],
 [3644, 71],
 [11, 3076],
 [623, 6, 322],
 [2, 173, 398, 15],
 [103, 2, 13, 544],
 [408, 12, 4596, 91, 6, 146, 1473, 4597],
 [5],
 [211, 350, 1014, 305, 44, 409, 1903],
 [2653, 1474, 191, 2, 23, 2653],
 [2, 1070, 1388, 1071, 3645, 4598, 1904, 35],
 [11, 474, 933, 9, 16, 559, 475, 1598, 161, 3646, 4599],
 [79, 34, 6, 866],
 [1310, 17, 589],
 [9, 10, 24, 272, 697, 4600]

In [11]:
longitud_maxima = max([len(secuencia) for secuencia in secuencias])
secuencias_rellenadas = pad_sequences(secuencias, maxlen=longitud_maxima)
secuencias_rellenadas.shape

(20001, 134)

In [29]:
type(tweets['clasificacion'][0])

numpy.int64

In [12]:
X_train, X_test, y_train, y_test = train_test_split(secuencias_rellenadas, tweets['clasificacion'], test_size=0.2, random_state=42)


In [42]:
modelo = Sequential()
modelo.add(Embedding(vocabulario_size, 128, input_length=longitud_maxima))
modelo.add(LSTM(128))
modelo.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [49]:
modelo.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
modelo.fit(X_train, y_train, epochs=16, batch_size=96,validation_data=(X_test,y_test))

Epoch 1/16
167/167 ━━━━━━━━━━━━━━━━━━━━ 80s 477ms/step - accuracy: 0.6939 - loss: 0.5761 - val_accuracy: 0.7391 - val_loss: 0.4998
Epoch 2/16
167/167 ━━━━━━━━━━━━━━━━━━━━ 83s 481ms/step - accuracy: 0.8392 - loss: 0.3675 - val_accuracy: 0.7938 - val_loss: 0.4358
Epoch 3/16
167/167 ━━━━━━━━━━━━━━━━━━━━ 80s 482ms/step - accuracy: 0.8877 - loss: 0.2764 - val_accuracy: 0.8330 - val_loss: 0.4062
Epoch 4/16
167/167 ━━━━━━━━━━━━━━━━━━━━ 82s 480ms/step - accuracy: 0.9171 - loss: 0.2184 - val_accuracy: 0.8423 - val_loss: 0.4167
Epoch 5/16
167/167 ━━━━━━━━━━━━━━━━━━━━ 81s 478ms/step - accuracy: 0.9406 - loss: 0.1573 - val_accuracy: 0.8535 - val_loss: 0.4204
Epoch 6/16
167/167 ━━━━━━━━━━━━━━━━━━━━ 82s 479ms/step - accuracy: 0.9490 - loss: 0.1404 - val_accuracy: 0.8618 - val_loss: 0.4289
Epoch 7/16
167/167 ━━━━━━━━━━━━━━━━━━━━ 82s 480ms/step - accuracy: 0.9578 - loss: 0.1143 - val_accuracy: 0.8613 - val_loss: 0.4904
Epoch 8/16
167/167 ━━━━━━━━━━━━━━━━━━━━ 80s 482ms/step - accuracy: 0.9622 - loss: 0